In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import inv

In [ ]:
asset_returns_orig = pd.read_csv('asset_returns.csv', index_col='Year', parse_dates=True)
asset_weights = pd.read_csv('asset_weights.csv', index_col='asset_class')
cols = ['Global Bonds (Unhedged)','Total US Bond Market','US Large Cap Growth',
            'US Large Cap Value','US Small Cap Growth','US Small Cap Value','Emerging Markets',
            'Intl Developed ex-US Market','Short Term Treasury']
asset_returns = asset_returns_orig[cols].dropna()
treasury_rate = asset_returns['Short Term Treasury']
asset_returns = asset_returns[cols[:-1]].astype(np.float).dropna()
asset_weights = asset_weights.loc[cols[:-1]]

We read a dataset called asset_returns and asset_weights and put it into a pandas dataframe. A pandas dataframe is basically a 2D array but there are a lot of utilities.
We can print out the average yearly returns and weights of each asset class below:

In [ ]:
asset_returns.mean()
asset_weights

Next we subtract the short term treasury rate from the asset class returns to obtain the relevant “excess returns” needed, generate the variance-covariance matrix of the excess returns. We subtract the rate because if you can't beat bonds then what are we doing this for. We then calculate the mean return and variance of the global market portfolio:

In [ ]:
excess_asset_returns = asset_returns.subtract(treasury_rate, axis=0)
cov = excess_asset_returns.cov()
global_return = excess_asset_returns.mean().multiply(asset_weights['weight'].values).sum()
market_var = np.matmul(asset_weights.values.reshape(len(asset_weights)).T,
                                       np.matmul(cov.values, asset_weights.values.reshape(len(asset_weights))))
print(f'The global market mean return is {global_return:.4f} and the variance is {market_var:.6}')
risk_aversion = global_return / market_var
print(f'The risk aversion parameter is {risk_aversion:.2f}')

Let’s write our first function which will help us reverse engineer the weights of a portfolio to obtain the Implied Equilibrium Return Vector.

In [ ]:
def implied_rets(risk_aversion, sigma, w):
    
    implied_rets = risk_aversion * sigma.dot(w).squeeze()
    
    return implied_rets
implied_equilibrium_returns = implied_rets(risk_aversion, cov, asset_weights)
implied_equilibrium_returns

Now we have some matrix of implied equilibrium returns but what if we want to go overweight on an asset class. Let's say we are feeling ambitious. We have 3 views:

View 1: ‘Emerging Markets’ will have an absolute excess return of 
9.25% (as opposed to the 7.62% equilibrium based value)

View 2: US Large Cap Growth and US Small Cap Growth will outperform US Large Cap Value and US Small Cap Value by 0.5% ((as opposed to the 1%-1.2% equilibrium based value)

View 3: ‘Intl Developed ex-US Market’ will have an absolute excess return of 5.5% (as opposed to the 6.31% equilibrium based value).

View 1 and 3 are examples of an absolute view, while view 2 is a relative view – FYI these relative type views tend to more closely represent the way most money/investment managers see the world and how they feel about different assets.

Now we have a view vector (Q) that is a 3 x 1 column vector. The uncertainty of the views results in a normally distributed error term vector with a mean of 0 and covariance matrix omega

In [ ]:
Q = np.array([0.0925, 0.005, 0.055])

We have a matrix P where we input our views. This is a parameter

In [ ]:
P = [[0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, .5, -.5, .5, -.5, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1]]